In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [4]:
def add_ad_to_car_dict(ad):
    car_url = ad
    try:
        response = requests.get(car_url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
    except:
        return
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 175000
for ad in all_ads[175000:200000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue
df_8 = pd.DataFrame.from_dict(car_dict, orient='index')
df_8.to_csv('Final_df_8.csv')

175000
175001
175002
175003
175004
175005
175006
175007
175008
175009
Completed 175010 of total 269381
175010
175011
175012
175013
175014
175015
175016
175017
175018
175019
Completed 175020 of total 269381
175020
175021
175022
175023
175024
175025
175026
175027
175028
175029
Completed 175030 of total 269381
175030
175031
175032
175033
175034
175035
175036
175037
175038
175039
Completed 175040 of total 269381
175040
175041
175042
175043
175044
175045
175046
175047
175048
175049
Completed 175050 of total 269381
175050
175051
175052
175053
175054
175055
175056
175057
175058
175059
Completed 175060 of total 269381
175060
175061
175062
175063
175064
175065
175066
175067
175068
175069
Completed 175070 of total 269381
175070
175071
175072
175073
175074
175075
175076
175077
175078
175079
Completed 175080 of total 269381
175080
175081
175082
175083
175084
175085
175086
175087
175088
175089
Completed 175090 of total 269381
175090
175091
175092
175093
175094
175095
175096
175097
175098
175099
Com

175798
175799
Completed 175800 of total 269381
175800
175801
175802
175803
175804
175805
175806
175807
175808
175809
Completed 175810 of total 269381
175810
175811
175812
175813
175814
175815
175816
175817
175818
175819
Completed 175820 of total 269381
175820
175821
175822
175823
175824
175825
175826
175827
175828
175829
Completed 175830 of total 269381
175830
175831
175832
175833
175834
175835
175836
175837
175838
175839
Completed 175840 of total 269381
175840
175841
175842
175843
175844
175845
175846
175847
175848
175849
Completed 175850 of total 269381
175850
175851
175852
175853
175854
175855
175856
175857
175858
175859
Completed 175860 of total 269381
175860
175861
175862
175863
175864
175865
175866
175867
175868
175869
Completed 175870 of total 269381
175870
175871
175872
175873
175874
175875
175876
175877
175878
175879
Completed 175880 of total 269381
175880
175881
175882
175883
175884
175885
175886
175887
175888
175889
Completed 175890 of total 269381
175890
175891
175892
17589

176592
176593
176594
176595
176596
176597
176598
176599
Completed 176600 of total 269381
176600
176601
176602
176603
176604
176605
176606
176607
176608
176609
Completed 176610 of total 269381
176610
176611
176612
176613
176614
176615
176616
176617
176618
176619
Completed 176620 of total 269381
176620
176621
176622
176623
176624
176625
176626
176627
176628
176629
Completed 176630 of total 269381
176630
176631
176632
176633
176634
176635
176636
176637
176638
176639
Completed 176640 of total 269381
176640
176641
176642
176643
176644
176645
176646
176647
176648
176649
Completed 176650 of total 269381
176650
176651
176652
176653
176654
176655
176656
176657
176658
176659
Completed 176660 of total 269381
176660
176661
176662
176663
176664
176665
176666
176667
176668
176669
Completed 176670 of total 269381
176670
176671
176672
176673
176674
176675
176676
176677
176678
176679
Completed 176680 of total 269381
176680
176681
176682
176683
176684
176685
176686
176687
176688
176689
Completed 176690 

Completed 177390 of total 269381
177390
177391
177392
177393
177394
177395
177396
177397
177398
177399
Completed 177400 of total 269381
177400
177401
177402
177403
177404
177405
177406
177407
177408
177409
Completed 177410 of total 269381
177410
177411
177412
177413
177414
177415
177416
177417
177418
177419
Completed 177420 of total 269381
177420
177421
177422
177423
177424
177425
177426
177427
177428
177429
Completed 177430 of total 269381
177430
177431
177432
177433
177434
177435
177436
177437
177438
177439
Completed 177440 of total 269381
177440
177441
177442
177443
177444
177445
177446
177447
177448
177449
Completed 177450 of total 269381
177450
177451
177452
177453
177454
177455
177456
177457
177458
177459
Completed 177460 of total 269381
177460
177461
177462
177463
177464
177465
177466
177467
177468
177469
Completed 177470 of total 269381
177470
177471
177472
177473
177474
177475
177476
177477
177478
177479
Completed 177480 of total 269381
177480
177481
177482
177483
177484
17748

178184
178185
178186
178187
178188
178189
Completed 178190 of total 269381
178190
178191
178192
178193
178194
178195
178196
178197
178198
178199
Completed 178200 of total 269381
178200
178201
178202
178203
178204
178205
178206
178207
178208
178209
Completed 178210 of total 269381
178210
178211
178212
178213
178214
178215
178216
178217
178218
178219
Completed 178220 of total 269381
178220
178221
178222
178223
178224
178225
178226
178227
178228
178229
Completed 178230 of total 269381
178230
178231
178232
178233
178234
178235
178236
178237
178238
178239
Completed 178240 of total 269381
178240
178241
178242
178243
178244
178245
178246
178247
178248
178249
Completed 178250 of total 269381
178250
178251
178252
178253
178254
178255
178256
178257
178258
178259
Completed 178260 of total 269381
178260
178261
178262
178263
178264
178265
178266
178267
178268
178269
Completed 178270 of total 269381
178270
178271
178272
178273
178274
178275
178276
178277
178278
178279
Completed 178280 of total 26938

...

196509
Completed 196510 of total 269381
196510
196511
196512
196513
196514
196515
196516
196517
196518
196519
Completed 196520 of total 269381
196520
196521
196522
196523
196524
196525
196526
196527
196528
196529
Completed 196530 of total 269381
196530
196531
196532
196533
196534
196535
196536
196537
196538
196539
Completed 196540 of total 269381
196540
196541
196542
196543
196544
196545
196546
196547
196548
196549
Completed 196550 of total 269381
196550
196551
196552
196553
196554
196555
196556
196557
196558
196559
Completed 196560 of total 269381
196560
196561
196562
196563
196564
196565
196566
196567
196568
196569
Completed 196570 of total 269381
196570
196571
196572
196573
196574
196575
196576
196577
196578
196579
Completed 196580 of total 269381
196580
196581
196582
196583
196584
196585
196586
196587
196588
196589
Completed 196590 of total 269381
196590
196591
196592
196593
196594
196595
196596
196597
196598
196599
Completed 196600 of total 269381
196600
196601
196602
196603
19660

197303
197304
197305
197306
197307
197308
197309
Completed 197310 of total 269381
197310
197311
197312
197313
197314
197315
197316
197317
197318
197319
Completed 197320 of total 269381
197320
197321
197322
197323
197324
197325
197326
197327
197328
197329
Completed 197330 of total 269381
197330
197331
197332
197333
197334
197335
197336
197337
197338
197339
Completed 197340 of total 269381
197340
197341
197342
197343
197344
197345
197346
197347
197348
197349
Completed 197350 of total 269381
197350
197351
197352
197353
197354
197355
197356
197357
197358
197359
Completed 197360 of total 269381
197360
197361
197362
197363
197364
197365
197366
197367
197368
197369
Completed 197370 of total 269381
197370
197371
197372
197373
197374
197375
197376
197377
197378
197379
Completed 197380 of total 269381
197380
197381
197382
197383
197384
197385
197386
197387
197388
197389
Completed 197390 of total 269381
197390
197391
197392
197393
197394
197395
197396
197397
197398
197399
Completed 197400 of tota

198101
198102
198103
198104
198105
198106
198107
198108
198109
Completed 198110 of total 269381
198110
198111
198112
198113
198114
198115
198116
198117
198118
198119
Completed 198120 of total 269381
198120
198121
198122
198123
198124
198125
198126
198127
198128
198129
Completed 198130 of total 269381
198130
198131
198132
198133
198134
198135
198136
198137
198138
198139
Completed 198140 of total 269381
198140
198141
198142
198143
198144
198145
198146
198147
198148
198149
Completed 198150 of total 269381
198150
198151
198152
198153
198154
198155
198156
198157
198158
198159
Completed 198160 of total 269381
198160
198161
198162
198163
198164
198165
198166
198167
198168
198169
Completed 198170 of total 269381
198170
198171
198172
198173
198174
198175
198176
198177
198178
198179
Completed 198180 of total 269381
198180
198181
198182
198183
198184
198185
198186
198187
198188
198189
Completed 198190 of total 269381
198190
198191
198192
198193
198194
198195
198196
198197
198198
198199
Completed 

198899
Completed 198900 of total 269381
198900
198901
198902
198903
198904
198905
198906
198907
198908
198909
Completed 198910 of total 269381
198910
198911
198912
198913
198914
198915
198916
198917
198918
198919
Completed 198920 of total 269381
198920
198921
198922
198923
198924
198925
198926
198927
198928
198929
Completed 198930 of total 269381
198930
198931
198932
198933
198934
198935
198936
198937
198938
198939
Completed 198940 of total 269381
198940
198941
198942
198943
198944
198945
198946
198947
198948
198949
Completed 198950 of total 269381
198950
198951
198952
198953
198954
198955
198956
198957
198958
198959
Completed 198960 of total 269381
198960
198961
198962
198963
198964
198965
198966
198967
198968
198969
Completed 198970 of total 269381
198970
198971
198972
198973
198974
198975
198976
198977
198978
198979
Completed 198980 of total 269381
198980
198981
198982
198983
198984
198985
198986
198987
198988
198989
Completed 198990 of total 269381
198990
198991
198992
198993
19899

199693
199694
199695
199696
199697
199698
199699
Completed 199700 of total 269381
199700
199701
199702
199703
199704
199705
199706
199707
199708
199709
Completed 199710 of total 269381
199710
199711
199712
199713
199714
199715
199716
199717
199718
199719
Completed 199720 of total 269381
199720
199721
199722
199723
199724
199725
199726
199727
199728
199729
Completed 199730 of total 269381
199730
199731
199732
199733
199734
199735
199736
199737
199738
199739
Completed 199740 of total 269381
199740
199741
199742
199743
199744
199745
199746
199747
199748
199749
Completed 199750 of total 269381
199750
199751
199752
199753
199754
199755
199756
199757
199758
199759
Completed 199760 of total 269381
199760
199761
199762
199763
199764
199765
199766
199767
199768
199769
Completed 199770 of total 269381
199770
199771
199772
199773
199774
199775
199776
199777
199778
199779
Completed 199780 of total 269381
199780
199781
199782
199783
199784
199785
199786
199787
199788
199789
Completed 199790 of tota